# Location Mapping
The raw data contains over a thousand different locations, a lot of them with very low occurrences. This can considerably reduce the quality of our model. We must filter our locations carefully without reducing the size of our dataset. To do so, we will associate every location to its closest population center.
\
\
This notebook goes through all location values and maps them to their closest population center. Population centers are defined as either:
\
\
Regions
- Quebec Administrative Regions (Estrie, Outaouais, Montérégie etc.)

In [151]:
import numpy as np
import pandas as pd

from geopy.distance import geodesic
from geopy.location import Location
from geopy.geocoders import Nominatim
from tqdm import tqdm

from IPython.display import display

## Useful Methods & Resources

In [137]:
geolocator = Nominatim(user_agent='housing-qc')

In [138]:
def find_coordinates(locations):
    location_dict = []
    unknown_locations = []

    for location in tqdm(locations):
        try:
            geocode = geolocator.geocode(location + ', QC')
        except:
            geocode = None
        
        if geocode is None:
            unknown_locations.append(location)
            print(location + ' is an unknown location')
        else:
            location_dict.append({'Name': location, 'Latitude': geocode.latitude, 'Longitude': geocode.longitude})

    return pd.DataFrame(location_dict), unknown_locations

In [139]:
# Dictionnary for unknown locations
unknown_locations_dict = {}
unknown_locations_dict["L'Ile Des Soeurs"] = "Ile des soeurs"
unknown_locations_dict["St-Joseph-De-Ham-Sud"] = "Ham-Sud"
unknown_locations_dict["St-Mathieu-De-Laprairie"] = "Saint-Mathieu"
unknown_locations_dict["St-Denis-sur-Mer"] = "Saint-Denis"
unknown_locations_dict["St-Isidore-De-Laprairie"] = "Saint-Isidore"
unknown_locations_dict["St-Stanislas-De-Champlain"] = "Saint-Denis"
unknown_locations_dict["St-Sebastien-De-Frontenac"] = "Saint-Sebastien"
unknown_locations_dict["St-Simon-De-Rimouski"] = "Saint-Simon"
unknown_locations_dict["Ste-Francoise-De-Lotbiniere"] = "Saint-Francoise"
unknown_locations_dict["St-Guillaume-D'Upton"] = "Saint-Guillaume"
unknown_locations_dict["St-Adelphe-De-Champlain"] = "Saint-Adelphe"

In [140]:
def fix_unknown_location(unknown_location: str):
    if unknown_location in unknown_locations_dict.keys():
        return unknown_locations_dict[unknown_location]
    elif "ND-" in unknown_location:
        return unknown_location.replace("ND-", "Notre-Dame-")
    elif "JC" in unknown_location:
        return unknown_location.replace("JC", "Jacques-Cartier")
    elif "St-" in unknown_location:
        return unknown_location.replace("St-", "Saint-")
    elif "Ste-" in unknown_location:
        return unknown_location.replace("Ste-", "Sainte-")
    elif "Sts-" in unknown_location:
        return unknown_location.replace("Sts-", "Saints-")
    else:
        return unknown_location

## Raw Locations

### Get Locations from Raw Listings

In [141]:
home_df = pd.read_csv('../data/raw/home_listings.csv')
condo_df = pd.read_csv('../data/raw/condo_listings.csv')
raw_df = pd.concat([home_df, condo_df], axis=0, ignore_index=True)

raw_locations = list(raw_df['location'].value_counts().to_dict().keys())
print('Raw Locations: ' + str(raw_locations[0]) + ', ' + str(raw_locations[1]) + ', ' + str(raw_locations[2]) + ' etc.')

Raw Locations: Gatineau, Trois-Rivières, Beauport etc.


### Get Raw Locations Coordinates

In [142]:
raw_location_coordinates, unknown_locations = find_coordinates(raw_locations)

 10%|▉         | 113/1167 [00:57<12:15,  1.43it/s]

Ste-Catherine-de-la-JC is an unknown location


 17%|█▋        | 195/1167 [01:38<08:33,  1.89it/s]

ND-De-L'Ile-Perrot is an unknown location


 26%|██▋       | 307/1167 [02:34<07:25,  1.93it/s]

...

 95%|█████████▌| 1113/1167 [09:26<00:25,  2.14it/s]

St-Denis-sur-Mer is an unknown location


 99%|█████████▉| 1158/1167 [09:48<00:04,  1.90it/s]

ND-De-Bonsecours is an unknown location


100%|██████████| 1167/1167 [09:53<00:00,  1.97it/s]


### Format Unknown Locations

In [143]:
fixed_locations = list(map(fix_unknown_location, unknown_locations))
unknown_locations_mapper = {fixed_locations[i]: unknown_locations[i] for i in range(len(unknown_locations))}

raw_location_coordinates2, unknown_locations2 = find_coordinates(fixed_locations)
if len(unknown_locations2) == 0:
    raw_location_coordinates2 = { unknown_locations_mapper.get(k, k): v for k, v in raw_location_coordinates2.items() }
    raw_location_coordinates.update(raw_location_coordinates2)

100%|██████████| 25/25 [00:12<00:00,  2.01it/s]


### Save Final Raw Locations Coordinates

In [145]:
raw_location_coordinates.to_csv('../data/processed/raw_location_coordinates.csv')
raw_location_coordinates.head()

,Name,Latitude,Longitude
0,Ste-Catherine-de-la-Jacques-Cartier,46.844381,-71.615023
1,Notre-Dame-De-L'Ile-Perrot,45.351663,-73.902969
2,Saint-Mathieu,45.312563,-73.518448
3,Saint-Isidore,46.585058,-71.090469
4,Saint-Magloire-De-Bellechasse,46.592524,-70.440777


## Regions

In [146]:
regions_df = pd.read_csv('../data/references/qc-administrative-regions.csv')
regions = dict(zip(regions_df['City'], regions_df['Region']))

display(regions_df.head())

,City,Region
0,Rimouski,Bas-Saint-Laurent
1,Rivière-du-Loup,Bas-Saint-Laurent
2,Matane,Bas-Saint-Laurent
3,Alma,Saguenay-Lac-Saint-Jean
4,Saguenay,Saguenay-Lac-Saint-Jean


### Get Region City Coordinates

In [148]:
region_city_coordinates, unknown_cities = find_coordinates(regions.keys())

region_city_coordinates.to_csv('../data/processed/region_city_coordinates.csv')
region_city_coordinates.head()

100%|██████████| 40/40 [00:19<00:00,  2.02it/s]


,Name,Latitude,Longitude
0,Rimouski,48.450155,-68.529968
1,Rivière-du-Loup,47.835816,-69.536802
2,Matane,48.846877,-67.529550
3,Alma,48.548887,-71.651459
4,Saguenay,48.405959,-71.069183


In [ ]:
def find_closest_location(latitude, longitude, locations: pd.DataFrame):
    distances = []

    for _, location in locations.iterrows():
        distances.append(geodesic((latitude, longitude), (location['Latitude'], location['Longitude'])))

    return locations.iloc[np.argmin(distances)]['Name']

In [ ]:
def build_location_mapper(locations: pd.DataFrame, reference_locations: pd.DataFrame):
    location_mapper = {}
    
    for _, location in locations.iterrows:
        closest_location = find_closest_location(location['Latitude'], location['Longitude'], reference_locations)
        location_mapper[location] = closest_location
        
    return location_mapper